# Calculation of detectable GRB event rate

In [494]:
from ler.rates import LeR
import numpy as np

## Probability of detection of GRBs

In [561]:
def pdet_grb(angle, distance, distance_cut=25422.742, duty_cycle=0.5, mask_duty_cycle=True, bool=True):
    """
    Function to calculate the probability of detection for a given angle and distance for GRB. Coefficients are based on the following condition,

    1. GRB jet, core angle <= 5 deg
    2. If angle <= 5 deg, distance <= distance_cut, then pdet = 1
    3. Luminosity(core)/Luminosity(30deg) >= 100, from GW170817

    Parameters
    ----------
    angle : numpy.ndarray
        Angle between the GRB jet and viewing angle in (rad)
    distance : numpy.ndarray
        Distance between the GRB source and the satellite in (Mpc)
    distance_cut : float
        Core angular size of the GRB jet in (rad)
        default is 25422.742 Mpc
    duty_cycle : float
        Duty cycle of detector(s)
        default is 0.5
    bool : bool
        if True, return absolute value of pdet
        if False, return the pdet value as boolean (with duty cycle applied)
    """

    # coefficients
    c = 2.9542496722537264
    a = 2513.274122871834
    # convert angle to degree
    angle = np.degrees(angle)

    # make sure that the input data is a numpy array
    # angle, distance = np.array([angle]).reshape(-1), np.array([distance]).reshape(-1)

    # angle should be less than 90 or equal to 90
    angle[angle > 90] = angle[angle > 90]%90

    if bool:
        # calculate the probability of detection
        pdet = abs((1 / (1 + (angle / 5)**c))* (40/distance)**2 * a/(4*np.pi))

        # find idx of angle <= 5 and distance <= distance_cut
        idx = (angle <= 5) & (distance <= distance_cut)

        # apply the condition
        pdet[idx] = 1

        if mask_duty_cycle:
            # apply the duty cycle
            # sample random numbers from 0 to 1 and check if it is less than the duty cycle
            num_ = np.random.rand(len(angle))
            mask_duty_cycle = num_ > duty_cycle
            pdet[mask_duty_cycle] = 0

        # return the pdet as boolean
        return (pdet>=1).astype(int)
    else:
        # return the probability of detection (absolute value)
        return abs((1 / (1 + (angle / 5)**c))* (40/distance)**2 * a/(4*np.pi))

In [562]:
# test
# pdet_calculator(angle=np.radians(5), distance=40, bool=False)/pdet_calculator(angle=np.radians(30), distance=40, bool=False)
angle = np.array([1, 30, 90])
angle = np.radians(angle)
distance = np.array([2500, 40, 40])
print(pdet_grb(angle, distance, mask_duty_cycle=False, bool=True))

[1 1 0]


* let's write out the function so that it can be used in with LeR
* consider 10% duty cycle

In [563]:
# let's write out the function so that it can be used in with LeR
def pdet_calculator(gw_param_dict, duty_cycle=0.1, mask_duty_cycle=True, bool=True):
    """
    Function to calculate the probability of detection for a given angle and distance for GRB. This is based on the following condition

    1. GRB jet, core angle <= 5 deg
    2. If angle <= 5 deg, distance <= distance_cut, then pdet = 1
    3. Luminosity(core)/Luminosity(30deg) > 100, from GW170817 

    Parameters
    ----------
    gw_param_dict : dict
        dictionary containing the parameters for the GW event
    """

    # get the angle and distance from the dictionary
    angle = gw_param_dict['theta_jn']
    distance = gw_param_dict['luminosity_distance']

    # calculate the probability of detection
    pdet = pdet_grb(angle, distance, duty_cycle=duty_cycle, mask_duty_cycle=mask_duty_cycle, bool=bool)

    # return the pdet
    return dict(pdet_net=pdet)

In [564]:
# test
gw_param_dict = {'theta_jn': np.radians(np.array([1, 30, 90])), 'luminosity_distance': np.array([2500, 40, 40])}
print(pdet_calculator(gw_param_dict, mask_duty_cycle=False, bool=True))

{'pdet_net': array([1, 1, 0])}


* initialize LeR with pdet calculator

In [568]:
ler = LeR(verbose=False, pdet_finder=pdet_calculator, event_type='BNS')

In [569]:
unlensed_param = ler.unlensed_cbc_statistics();

unlensed params will be store in ./ler_data/unlensed_param.json
chosen batch size = 50000 with total size = 100000
There will be 2 batche(s)
Batch no. 1
sampling gw source params...
calculating pdet...
Batch no. 2
sampling gw source params...
calculating pdet...
saving all unlensed_params in ./ler_data/unlensed_param.json...


In [570]:
_, unlensed_param_detectable = ler.unlensed_rate()

getting unlensed_params from json file ./ler_data/unlensed_param.json...
given detectability_condition == 'pdet'
total unlensed rate (yr^-1) (with step function): 296.0754841959819
number of simulated unlensed detectable events: 286
number of all simulated unlensed events: 100000
storing detectable unlensed params in ./ler_data/unlensed_param_detectable.json


* we have ~300 GRBs (detectable) per year
* let's quickly calculate for GW from BNS too 

In [571]:
ler = LeR(verbose=False, event_type='BNS')
unlensed_param = ler.unlensed_cbc_statistics();
_, unlensed_param_detectable = ler.unlensed_rate()

unlensed params will be store in ./ler_data/unlensed_param.json
chosen batch size = 50000 with total size = 100000
There will be 2 batche(s)
Batch no. 1
sampling gw source params...
calculating snrs...
Batch no. 2
sampling gw source params...
calculating snrs...
saving all unlensed_params in ./ler_data/unlensed_param.json...
getting unlensed_params from json file ./ler_data/unlensed_param.json...
given detectability_condition == 'step_function'
total unlensed rate (yr^-1) (with step function): 3.1056868971606497
number of simulated unlensed detectable events: 3
number of all simulated unlensed events: 100000
storing detectable unlensed params in ./ler_data/unlensed_param_detectable.json


* Around 3 BNS GW events (detectable) per year, as compared to 300 GRBs.